In [1]:
using Distributed
addprocs(5)
nprocs()

6

In [2]:
import Pkg
Pkg.add("CSV")
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("DataStructures")
Pkg.add("TextAnalysis")

#import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
#using JuliaAcademyData; activate("Parallel_Computing")
@everywhere using DataFrames
@everywhere using CSV
@everywhere using TextAnalysis, DataStructures
import Dates

In [4]:
using Dates
startTime = Dates.Time(Dates.now()) 

23:45:39.609

In [5]:
import CSV
@everywhere df = DataFrame(CSV.File("/home/joelkik/HPC/Project/Data/amazon_reviews_us_Apparel_v1_00.tsv"))

┌ Warning: thread = 1 warning: only found 14 / 15 columns around data row: 29538. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 1 warning: parsed expected 15 columns, but didn't reach end of line around data row: 49395. Parsing extra columns and widening final columnset
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:579
┌ Warning: thread = 1 warning: only found 15 / 22 columns around data row: 49396. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 1 warning: only found 15 / 22 columns around data row: 49397. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 1 warning: only found 15 / 22 columns around data row: 49398. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 1 war

In [6]:
@everywhere function calTermAndDocTerm(docs)
    #println("ThreadId ",Threads.threadid())
    termDict=Dict()
    docTerm=Dict()
    docTermCount=[]
    for i in 1:length(docs)
        dt=Dict()
        for t in docs[i].tokens
            if haskey(termDict, t)
                termDict[t]=termDict[t]+1
            else
                termDict[t]=1
            end
            if haskey(docTerm, t)
                push!(docTerm[t], i)
            else
                docTerm[t]=Set(i)
            end
            if haskey(dt, t)
                dt[t]=dt[t]+1
            else
                dt[t]=1
            end
        end
        push!(docTermCount, dt)
    end
    return [docTerm, docTermCount]
end

@everywhere function preprocessDocsGetDocTerm( st, ed)
    docs=[]
    for i in st:ed
        if typeof(df.review_body[i])==String
            sd=TokenDocument(df.review_body[i])
            prepare!(sd, strip_stopwords)
            prepare!(sd, strip_non_letters)
            push!(docs, sd)
        end
    end
    return calTermAndDocTerm(docs)
end

In [7]:
#@everywhere df
totalDocs=nrow(df)
n = floor(Int, totalDocs/4)
#remotecall_fetch(()->preprocessDocsGetDocTerm(df, 1, n), 2);
dT1 = @spawnat 5 preprocessDocsGetDocTerm(1, n)

dT2 = @spawnat 2 preprocessDocsGetDocTerm(n+1, n*2)
#println("2")
dT3 = @spawnat 3 preprocessDocsGetDocTerm(n*2+1, n*3)
#println("3")
dT4 = @spawnat 4 preprocessDocsGetDocTerm(n*3+1, totalDocs)
wait(dT1)
wait(dT2)
wait(dT3)
wait(dT4)


Future(4, 1, 100, nothing)

In [8]:
@everywhere docTerms=Dict()

@everywhere function mergeDocTerm(d)
    for (k,v) in d
        if haskey(docTerms, k)
            global docTerms[k]+=length(v)
        else
            global docTerms[k]=length(v)
        end
    end
end

res1 = fetch(dT1)
res2 = fetch(dT2)
res3 = fetch(dT3)
res4 = fetch(dT4)
#res5 = fetch(dT5)
#res6 = fetch(dT6)
#res7 = fetch(dT7)
#res8 = fetch(dT8)

mergeDocTerm(res1[1])
mergeDocTerm(res2[1])
mergeDocTerm(res3[1])
mergeDocTerm(res4[1])
#mergeDocTerm(res5[1])
#mergeDocTerm(res6[1])
#mergeDocTerm(res7[1])
#mergeDocTerm(res8[1])
print(haskey(docTerms, "Excellent"))
#println(docTerms)

true

In [9]:
@everywhere  IDF=Dict()
for (k,v) in docTerms
    global IDF[k]=log2(totalDocs/length(v))
end
#@everywhere IDF
#print(IDF)

In [10]:
@everywhere function calTfIdf( docTermCount)
    tf_idf=[]
    #println(docTermCount)
    for i in 1:length(docTermCount)
        dt=Dict()
        for (k,v) in docTermCount[i]
                dt[k]=v*IDF[k]
        end
        push!(tf_idf, dt)
    end
    return tf_idf
end

In [14]:
tfIdf1 = @spawnat 1 calTfIdf(res1[2])
tfIdf2 = @spawnat 2 calTfIdf(res2[2])
tfIdf3 = @spawnat 3 calTfIdf(res3[2])
tfIdf4 = @spawnat 4 calTfIdf(res4[2])

wait(tfIdf1)
wait(tfIdf2)
wait(tfIdf3)
wait(tfIdf4)

#println("TFIDF ", fetch(tfIdf4))

LoadError: On worker 2:
KeyError: key "pole" not found
Stacktrace:
 [1] [0m[1mgetindex[22m
[90m   @ [39m[90m./[39m[90;4mdict.jl:482[0m
 [2] [0m[1mcalTfIdf[22m
[90m   @ [39m[90m./[39m[90;4mIn[10]:7[0m
 [3] [0m[1m#11[22m
[90m   @ [39m[90m/tmp/spack/spack-stage/spack-stage-julia-1.6.1-h2k7dmirfmch64svvlxtxn652ylsfjx4/spack-src/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mmacros.jl:87[0m
 [4] [0m[1m#103[22m
[90m   @ [39m[90m/tmp/spack/spack-stage/spack-stage-julia-1.6.1-h2k7dmirfmch64svvlxtxn652ylsfjx4/spack-src/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mprocess_messages.jl:274[0m
 [5] [0m[1mrun_work_thunk[22m
[90m   @ [39m[90m/tmp/spack/spack-stage/spack-stage-julia-1.6.1-h2k7dmirfmch64svvlxtxn652ylsfjx4/spack-src/usr/share/julia/stdlib/v1.6/Distributed/src/[39m[90;4mprocess_messages.jl:63[0m
 [6] [0m[1mrun_work_thunk[22m
[90m   @ [39m[90m/tmp/spack/spack-stage/spack-stage-julia-1.6.1-h2k7dmirfmch64svvlxtxn652ylsfjx4/s

In [12]:
endTime = Dates.Time(Dates.now()) 

00:00:06.845

In [13]:
println("Time Taken: ", endTime-startTime)

Time Taken: -85532764000000 nanoseconds
